In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import machine learning modules
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



In [3]:
data = pd.read_csv('Pro_Algo_Sample.csv')

In [4]:
data.head()

,POS_UNIQUE_ID,POS_CONTROLLER,POS_CONTROLLER_DESCRIPTION,FISCYEAR,YFISCQRT,FISCPER,YFISCWEEK,CALDAY,END_TIME_STAMP,MATERIAL,...,POS_Qty_SUoM,REGISTER,SALES_ITEM_TYPE_DESCRIPTION,Sales_UoM,XREFERENCE_NUMBER,FLAG,prov,prov2,DISTRICT,MARKET
0,7274-20180309-02508-090,3V,Sp. Srvc. Dsk VPOS,2018,20181,2018002,201806,20180309,230445,1000793158,...,3.0,90,Special Order Sales,CS,1018762702094863,C,Online,Online,D-Ecomm,Online
1,7274-20180919-09594-090,3V,Sp. Srvc. Dsk VPOS,2018,20183,2018008,201834,20180919,104641,1001045206,...,3.0,90,Special Order Sales,CS,1019323483279227,C,Online,Online,D-Ecomm,Online
2,7274-20181008-04560-090,3V,Sp. Srvc. Dsk VPOS,2018,20183,2018009,201837,20181008,84531,1001122075,...,2.0,90,Special Order Sales,CS,1018976413454509,C,Online,Online,D-Ecomm,Online
3,7274-20180906-05830-090,3V,Sp. Srvc. Dsk VPOS,2018,20183,2018008,201832,20180906,115508,1001053164,...,1.0,90,Special Order Sales,EA,1019252039861004,P,Online,Online,D-Ecomm,Online
4,7274-20180824-03500-090,3V,Sp. Srvc. Dsk VPOS,2018,20183,2018007,201830,20180824,30229,1001125265,...,2.0,90,Special Order Sales,EA,1018955529645589,C,Online,Online,D-Ecomm,Online


In [5]:
data.describe()

,FISCYEAR,YFISCQRT,FISCPER,YFISCWEEK,CALDAY,END_TIME_STAMP,MATERIAL,YCLASS,YSUBCLASS,PLANT,POS_Amt,POS_Qty_SUoM,REGISTER,XREFERENCE_NUMBER
count,100.0,100.000000,1.000000e+02,100.00000,1.000000e+02,100.000000,1.000000e+02,100.00000,100.000000,100.0,100.000000,100.000000,100.0,1.000000e+02
mean,2018.0,20181.940000,2.018005e+06,201819.55000,2.018060e+07,140201.670000,1.000800e+09,2097.80000,10591.950000,7274.0,191.883900,3.130000,90.0,1.017301e+15
std,0.0,0.826579,2.616266e+00,11.42598,2.641013e+02,48107.418268,2.554802e+05,36.26641,232.328458,0.0,198.716729,6.097946,0.0,3.205803e+12
min,2018.0,20181.000000,2.018001e+06,201801.00000,2.018013e+07,4833.000000,1.000108e+09,2005.00000,10030.000000,7274.0,-50.000000,-1.000000,90.0,1.002414e+15
25%,2018.0,20181.000000,2.018003e+06,201810.75000,2.018041e+07,114781.250000,1.000724e+09,2060.00000,10366.000000,7274.0,63.395000,1.000000,90.0,1.017063e+15
50%,2018.0,20182.000000,2.018005e+06,201819.00000,2.018061e+07,144576.500000,1.000794e+09,2118.00000,10710.500000,7274.0,128.300000,1.000000,90.0,1.018438e+15
75%,2018.0,20183.000000,2.018007e+06,201830.00000,2.018082e+07,172240.000000,1.001029e+09,2122.00000,10761.000000,7274.0,241.250000,2.000000,90.0,1.019114e+15
max,2018.0,20183.000000,2.018009e+06,201839.00000,2.018102e+07,234619.000000,1.001201e+09,2124.00000,10793.000000,7274.0,1004.000000,35.000000,90.0,1.019596e+15


In [6]:
data.columns

Index(['POS_UNIQUE_ID', 'POS_CONTROLLER', 'POS_CONTROLLER_DESCRIPTION',
       'FISCYEAR', 'YFISCQRT', 'FISCPER', 'YFISCWEEK', 'CALDAY',
       'END_TIME_STAMP', 'MATERIAL', 'MATERIAL_DESCRIPTION',
       'MATERIAL_YDEPT_DESCRIPTION', 'EXT_YMDEPT_DESCRIPTION',
       'YCLASS_DESCRIPTION', 'YSUBCLASS_DESCRIPTION', 'YCLASS', 'YSUBCLASS',
       'PLANT', 'POS_Amt', 'POS_Qty_SUoM', 'REGISTER',
       'SALES_ITEM_TYPE_DESCRIPTION', 'Sales_UoM', 'XREFERENCE_NUMBER', 'FLAG',
       'prov', 'prov2', 'DISTRICT', 'MARKET'],
      dtype='object')

In [7]:
data.groupby(['FLAG'])['Sales_UoM'].count()

FLAG
C    86
P    14
Name: Sales_UoM, dtype: int64

In [9]:
pro = []
for i in range(len(data)):
    if data['FLAG'][i] == 'C':
        pro.append(0)
    else:
        pro.append(1)

In [12]:
sum(pro)

14

In [13]:
data['pro'] = pro

In [14]:
cat_features = list(data.columns[data.dtypes == 'object'])
cat_features

['POS_UNIQUE_ID',
 'POS_CONTROLLER',
 'POS_CONTROLLER_DESCRIPTION',
 'MATERIAL_DESCRIPTION',
 'MATERIAL_YDEPT_DESCRIPTION',
 'EXT_YMDEPT_DESCRIPTION',
 'YCLASS_DESCRIPTION',
 'YSUBCLASS_DESCRIPTION',
 'SALES_ITEM_TYPE_DESCRIPTION',
 'Sales_UoM',
 'FLAG',
 'prov',
 'prov2',
 'DISTRICT',
 'MARKET']

In [15]:
cat_dict = {}

for col in cat_features:
    cat_dict[col] = {}
    count = 0
    for label in list(data[col].unique()):
        cat_dict[col][label] = count
        count += 1
    
def convert_cat(col, label):
    return (cat_dict[col][label])

for col in cat_features:
    data[col] = data[col].apply(lambda x: convert_cat(col,x))

In [16]:
data.head()

,POS_UNIQUE_ID,POS_CONTROLLER,POS_CONTROLLER_DESCRIPTION,FISCYEAR,YFISCQRT,FISCPER,YFISCWEEK,CALDAY,END_TIME_STAMP,MATERIAL,...,REGISTER,SALES_ITEM_TYPE_DESCRIPTION,Sales_UoM,XREFERENCE_NUMBER,FLAG,prov,prov2,DISTRICT,MARKET,pro
0,0,0,0,2018,20181,2018002,201806,20180309,230445,1000793158,...,90,0,0,1018762702094863,0,0,0,0,0,0
1,1,0,0,2018,20183,2018008,201834,20180919,104641,1001045206,...,90,0,0,1019323483279227,0,0,0,0,0,0
2,2,0,0,2018,20183,2018009,201837,20181008,84531,1001122075,...,90,0,0,1018976413454509,0,0,0,0,0,0
3,3,0,0,2018,20183,2018008,201832,20180906,115508,1001053164,...,90,0,1,1019252039861004,1,0,0,0,0,1
4,4,0,0,2018,20183,2018007,201830,20180824,30229,1001125265,...,90,0,1,1018955529645589,0,0,0,0,0,0


In [18]:
data.columns

Index(['POS_UNIQUE_ID', 'POS_CONTROLLER', 'POS_CONTROLLER_DESCRIPTION',
       'FISCYEAR', 'YFISCQRT', 'FISCPER', 'YFISCWEEK', 'CALDAY',
       'END_TIME_STAMP', 'MATERIAL', 'MATERIAL_DESCRIPTION',
       'MATERIAL_YDEPT_DESCRIPTION', 'EXT_YMDEPT_DESCRIPTION',
       'YCLASS_DESCRIPTION', 'YSUBCLASS_DESCRIPTION', 'YCLASS', 'YSUBCLASS',
       'PLANT', 'POS_Amt', 'POS_Qty_SUoM', 'REGISTER',
       'SALES_ITEM_TYPE_DESCRIPTION', 'Sales_UoM', 'XREFERENCE_NUMBER', 'FLAG',
       'prov', 'prov2', 'DISTRICT', 'MARKET', 'pro'],
      dtype='object')

In [19]:
X = data[['POS_UNIQUE_ID', 'POS_CONTROLLER', 'POS_CONTROLLER_DESCRIPTION',
       'FISCYEAR', 'YFISCQRT', 'FISCPER', 'YFISCWEEK', 'CALDAY',
       'END_TIME_STAMP', 'MATERIAL', 'MATERIAL_DESCRIPTION',
       'MATERIAL_YDEPT_DESCRIPTION', 'EXT_YMDEPT_DESCRIPTION',
       'YCLASS_DESCRIPTION', 'YSUBCLASS_DESCRIPTION', 'YCLASS', 'YSUBCLASS',
       'PLANT', 'POS_Amt', 'POS_Qty_SUoM', 'REGISTER',
       'SALES_ITEM_TYPE_DESCRIPTION', 'Sales_UoM', 'XREFERENCE_NUMBER',
       'prov', 'prov2', 'DISTRICT', 'MARKET']]
y = data['pro']

In [25]:
def ml_classification_tests(X, y, num_cols=X.shape[1]):
    
    # Split into training and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    #------------------------------------------------------------------------
    
    
    #------------------------------------------------------------------------
    
    # RANDOM FORESTS
    print('\nRandom Forests')
    
    # Number of features to consider at every split
    max_features = [1, 2, 3, 4, 5]
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4, 6, 9]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}    
    
    # Instantiate a Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=250)
    
    # Instantiate RandomizedSearchCV
    rf_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, cv = 5, n_jobs = -1, n_iter=12)
    
    # Fit the random search model
    rf_random.fit(X_train, y_train)
    
    # Print the tuned parameters and score
    print("Tuned Random Forest Parameters: {}".format(rf_random.best_params_))
    print("Best Random Forest score: {}".format(rf_random.best_score_))
    
    # Predict the labels of the test set: y_pred
    y_pred = rf_random.predict(X_test)

    # Compute and print the confusion matrix and classification report
    print('Confusion Matrix:', confusion_matrix(y_test, y_pred))
    print('Classification Report:', classification_report(y_test, y_pred))

    #------------------------------------------------------------------------

In [27]:
import imblearn
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X,y)
print(sorted(Counter(y_resampled).items()))

[(0, 86), (1, 86)]


In [28]:
ml_classification_tests(X_resampled, y_resampled)


Random Forests
Tuned Random Forest Parameters: {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 3, 'max_depth': None, 'bootstrap': True}
Best Random Forest score: 0.8914728682170543
Confusion Matrix: [[21  1]
 [ 0 21]]
Classification Report:               precision    recall  f1-score   support

           0       1.00      0.95      0.98        22
           1       0.95      1.00      0.98        21

   micro avg       0.98      0.98      0.98        43
   macro avg       0.98      0.98      0.98        43
weighted avg       0.98      0.98      0.98        43



/home/andrea/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [24]:
X.describe()

,POS_UNIQUE_ID,POS_CONTROLLER,POS_CONTROLLER_DESCRIPTION,FISCYEAR,YFISCQRT,FISCPER,YFISCWEEK,CALDAY,END_TIME_STAMP,MATERIAL,...,POS_Amt,POS_Qty_SUoM,REGISTER,SALES_ITEM_TYPE_DESCRIPTION,Sales_UoM,XREFERENCE_NUMBER,prov,prov2,DISTRICT,MARKET
count,100.000000,100.0,100.0,100.0,100.000000,1.000000e+02,100.00000,1.000000e+02,100.000000,1.000000e+02,...,100.000000,100.000000,100.0,100.000000,100.000000,1.000000e+02,100.0,100.0,100.0,100.0
mean,49.500000,0.0,0.0,2018.0,20181.940000,2.018005e+06,201819.55000,2.018060e+07,140201.670000,1.000800e+09,...,191.883900,3.130000,90.0,0.020000,0.970000,1.017301e+15,0.0,0.0,0.0,0.0
std,29.011492,0.0,0.0,0.0,0.826579,2.616266e+00,11.42598,2.641013e+02,48107.418268,2.554802e+05,...,198.716729,6.097946,0.0,0.140705,0.171447,3.205803e+12,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,2018.0,20181.000000,2.018001e+06,201801.00000,2.018013e+07,4833.000000,1.000108e+09,...,-50.000000,-1.000000,90.0,0.000000,0.000000,1.002414e+15,0.0,0.0,0.0,0.0
25%,24.750000,0.0,0.0,2018.0,20181.000000,2.018003e+06,201810.75000,2.018041e+07,114781.250000,1.000724e+09,...,63.395000,1.000000,90.0,0.000000,1.000000,1.017063e+15,0.0,0.0,0.0,0.0
50%,49.500000,0.0,0.0,2018.0,20182.000000,2.018005e+06,201819.00000,2.018061e+07,144576.500000,1.000794e+09,...,128.300000,1.000000,90.0,0.000000,1.000000,1.018438e+15,0.0,0.0,0.0,0.0
75%,74.250000,0.0,0.0,2018.0,20183.000000,2.018007e+06,201830.00000,2.018082e+07,172240.000000,1.001029e+09,...,241.250000,2.000000,90.0,0.000000,1.000000,1.019114e+15,0.0,0.0,0.0,0.0
max,99.000000,0.0,0.0,2018.0,20183.000000,2.018009e+06,201839.00000,2.018102e+07,234619.000000,1.001201e+09,...,1004.000000,35.000000,90.0,1.000000,1.000000,1.019596e+15,0.0,0.0,0.0,0.0


In [30]:
from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso, RandomizedLasso)
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [31]:

names = X.keys()
 
#use linear regression as the model
rfc = RandomForestClassifier(n_estimators=250,
                            min_samples_split = 5,
                            min_samples_leaf = 1,
                            max_features = 0.25,
                            max_depth = 40,
                            bootstrap = False)
#rank all features, i.e continue the elimination until the last one
rfe = RFE(rfc, n_features_to_select=1)
rfe.fit(X_resampled,y_resampled)
 
print ("Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), names)))

Features sorted by their rank:
[(1, 'POS_Amt'), (2, 'END_TIME_STAMP'), (3, 'XREFERENCE_NUMBER'), (4, 'MATERIAL'), (5, 'MATERIAL_DESCRIPTION'), (6, 'YSUBCLASS'), (7, 'CALDAY'), (8, 'POS_UNIQUE_ID'), (9, 'YFISCWEEK'), (10, 'YCLASS'), (11, 'YSUBCLASS_DESCRIPTION'), (12, 'POS_Qty_SUoM'), (13, 'FISCPER'), (14, 'YCLASS_DESCRIPTION'), (15, 'EXT_YMDEPT_DESCRIPTION'), (16, 'MATERIAL_YDEPT_DESCRIPTION'), (17, 'YFISCQRT'), (18, 'Sales_UoM'), (19, 'SALES_ITEM_TYPE_DESCRIPTION'), (20, 'PLANT'), (21, 'REGISTER'), (22, 'FISCYEAR'), (23, 'POS_CONTROLLER_DESCRIPTION'), (24, 'POS_CONTROLLER'), (25, 'prov'), (26, 'prov2'), (27, 'DISTRICT'), (28, 'MARKET')]


In [32]:
data.groupby(['FLAG'])['POS_Amt'].mean()

FLAG
0    178.111279
1    276.487143
Name: POS_Amt, dtype: float64

In [35]:
data.groupby(['FLAG'])['END_TIME_STAMP'].min()

FLAG
0      4833
1    104037
Name: END_TIME_STAMP, dtype: int64